In [ ]:
import os
import cv2
import tifffile as tifi
import pandas as pd
import numpy as np
import rasterio
from glob import glob
from tqdm import tqdm
from shutil import copyfile

from matplotlib import pyplot as plt

from code_base.utils.mask import rle_decode, overlay_mask, rle_decode_multichannel_mask, rle_encode_less_memory
from code_base.utils.other import imread_rgb
from code_base.constants import CLASSES

# Data

In [ ]:
# row_df = pd.read_pickle("/home/vova/data/exps/hubmap_2022/denis_add_data/HuBMAP_Pseudo_2022-08-03 233641.992715.pkl")
# row_df_2 = pd.read_pickle("/home/vova/data/exps/hubmap_2022/denis_add_data/HuBMAP_Pseudo_2022-08-06 101623.892094.pkl")
row_df = pd.read_csv("/home/vova/data/exps/hubmap_2022/denis_add_data/v2/train.csv")

In [ ]:
# intersection = set(row_df["id"]) & set(row_df_2["id"])
# row_df = pd.concat([row_df[~row_df["id"].isin(intersection)], row_df_2], axis=0).reset_index(drop=True)

In [ ]:
# for item in row_df["mask"]:
#     print(np.unique(item))

In [ ]:
# row_df_normalized = {
#     "id":[],
#     "rle": [],
#     "organ": [],
#     "img_width": [],
#     "img_height": []
# }
# for i in range(len(row_df)):
#     new_id = os.path.splitext(row_df.iloc[i].id)[0]
#     new_im = row_df["image"].iloc[i][0]
#     new_mask = row_df["mask"].iloc[i][0]
#     new_organ = row_df["organ"].iloc[i]
    
#     tifi.imwrite(
#         f"/home/vova/data/exps/hubmap_2022/denis_add_data/v2/images/{new_id}.tiff",
#         new_im
#     )
#     mask_rle = rle_encode_less_memory(new_mask)
    
#     row_df_normalized["id"].append(new_id)
#     row_df_normalized["rle"].append(mask_rle)
#     row_df_normalized["organ"].append(new_organ)
#     row_df_normalized["img_height"].append(new_im.shape[0])
#     row_df_normalized["img_width"].append(new_im.shape[1])
# row_df_normalized = pd.DataFrame(row_df_normalized)

# row_df_normalized.to_csv("/home/vova/data/exps/hubmap_2022/denis_add_data/v2/train.csv", index=False)

In [ ]:
idx = 20

img = imread_rgb(
    f"/home/vova/data/exps/hubmap_2022/denis_add_data/v2/images/{row_df.id.iloc[idx]}.tiff", 
)
mask = rle_decode_multichannel_mask(
    None,
    row_df.rle.iloc[idx],
    (row_df.img_width.iloc[idx], row_df.img_height.iloc[idx]),
    one_channel=True
)

plt.figure(figsize=(20,20))
plt.imshow(overlay_mask(img,mask, color_id=1))
plt.show()

In [ ]:
row_df

# V3 (Adding Kidney and Colon)

In [ ]:
df = pd.read_csv("/home/vova/data/exps/hubmap_2022/denis_add_data/v2/train.csv")

In [ ]:
df.head()

In [ ]:
all_masks = glob("/home/vova/data/exps/hubmap_2022/denis_add_data/kidneycolon/*.png")

In [ ]:
all_images = (
    glob("/home/vova/data/exps/hubmap_2022/gtx_colon/images/*.tif") + 
    glob("/home/vova/data/exps/hubmap_2022/gtx_kidney/images/*.tif")
)

In [ ]:
# -5 refers to delete `_mask` at the end
masks_basenames = [os.path.splitext(os.path.basename(el))[0][:-5] for el in all_masks]
images_basenames = [os.path.splitext(os.path.basename(el))[0] for el in all_images]

In [ ]:
set(masks_basenames) - set(images_basenames)

In [ ]:
set(images_basenames) - set(masks_basenames)

In [ ]:
# row_df_normalized = {
#     "id":[],
#     "rle": [],
#     "organ": [],
#     "img_width": [],
#     "img_height": []
# }
# for image_fname in tqdm(images_basenames):
#     new_id = os.path.splitext(os.path.basename(image_fname))[0]
#     mask_fname = f"/home/vova/data/exps/hubmap_2022/denis_add_data/kidneycolon/{new_id}_mask.png"
#     new_organ = "kidney" if "kidney" in new_id else "largeintestine"
    
#     new_mask = cv2.imread(mask_fname, -1)
#     assert len(np.unique(new_mask)) == 2, mask_fname
#     mask_rle = rle_encode_less_memory(new_mask > 0.5)
    
#     row_df_normalized["id"].append(new_id)
#     row_df_normalized["rle"].append(mask_rle)
#     row_df_normalized["organ"].append(new_organ)
#     row_df_normalized["img_height"].append(new_mask.shape[0])
#     row_df_normalized["img_width"].append(new_mask.shape[1])
# row_df_normalized = pd.DataFrame(row_df_normalized)

# df = pd.concat([df,row_df_normalized]).reset_index(drop=True)

# df.to_csv("/home/vova/data/exps/hubmap_2022/denis_add_data/v3/train.csv", index=False)

In [ ]:
idx = -1

img = imread_rgb(
    f"/home/vova/data/exps/hubmap_2022/denis_add_data/v3/images/{df.id.iloc[idx]}.tiff", 
    backend="tifi"
)
mask = rle_decode_multichannel_mask(
    None,
    df.rle.iloc[idx],
    (df.img_width.iloc[idx], df.img_height.iloc[idx]),
    one_channel=True
)

plt.figure(figsize=(20,20))
plt.imshow(overlay_mask(img,mask, color_id=1))
plt.show()

In [ ]:
df = pd.read_csv("/home/vova/data/exps/hubmap_2022/denis_add_data/v3/train.csv")

In [ ]:
df.shape

In [ ]:
len(glob("/home/vova/data/exps/hubmap_2022/denis_add_data/v3/images/*.tiff"))

# V4 (new data and new Denis algorithm)

In [ ]:
def define_organ(input):
    input_base = os.path.basename(input)
    for organ in CLASSES:
        if organ in input_base:
            return organ
    raise RuntimeError(f"Not found organ for {input}")

In [ ]:
# image_pathes = glob("/home/vova/data/exps/hubmap_2022/**/*.t*", recursive=True)
image_pathes = (
    glob("/home/vova/data/exps/hubmap_2022/gtx_*/**/*.t*", recursive=True) 
    # glob("/home/vova/data/exps/hubmap_2022/domain_adaptation_data/**/*.t*", recursive=True) 
)
image_base_2_full = {}
for el in image_pathes:
    base = os.path.splitext(os.path.basename(el))[0]
    if base in image_base_2_full:
        image_base_2_full[base].append(el)
    else:
        image_base_2_full[base] = [el]

In [ ]:
mask_pathes = glob("/home/vova/data/exps/hubmap_2022/gtx_new_all_organs/masks/*")
mask_base_2_full = {}
for el in mask_pathes:
    base = os.path.splitext(os.path.basename(el))[0][:-5]
    if base in mask_base_2_full:
        mask_base_2_full[base].append(el)
    else:
        mask_base_2_full[base] = [el]

In [ ]:
v3_pseudo = pd.read_csv("/home/vova/data/exps/hubmap_2022/denis_add_data/v3/train.csv")
set(v3_pseudo["id"]) - set(mask_base_2_full.keys())

In [ ]:
image_full_2_mask_full = {}
for el in mask_base_2_full:
    if len(mask_base_2_full[el]) > 1:
        raise RuntimeError("several masks")
    else:
        mask_full = mask_base_2_full[el][0]
    if el not in image_base_2_full:
        print(f"Missing image {el}")
        continue
    # if len(image_base_2_full[el]) > 1:
    #     print(f"several images: {image_base_2_full[el]}")
    #     image_full = image_base_2_full[el][0]
    # else:
    image_full = image_base_2_full[el][0]
    image_full_2_mask_full[image_full] = mask_full

In [ ]:
os.makedirs("/home/vova/data/exps/hubmap_2022/denis_add_data/v4")
os.makedirs("/home/vova/data/exps/hubmap_2022/denis_add_data/v4/images")

In [ ]:
row_df_normalized = {
    "id":[],
    "rle": [],
    "organ": [],
    "img_width": [],
    "img_height": []
}
for im_name, mask_name in tqdm(image_full_2_mask_full.items()):
    new_id = os.path.splitext(os.path.basename(im_name))[0]
    new_organ = define_organ(im_name)
    
    new_mask = cv2.imread(mask_name, -1)
    img = imread_rgb(
        im_name,
        backend="tifi"
    )
    assert len(np.unique(new_mask)) == 2, mask_fname
    if new_mask.shape[:2] != img.shape[:2]:
        print(f"Shape inconsistencey: mask path: {mask_name}. image path: {im_name}")
        print("Trying to take another image ...")
        im_name = image_base_2_full[new_id][1]
        img = imread_rgb(
            im_name,
            backend="tifi"
        )
    assert new_mask.shape[:2] == img.shape[:2], f"Shape inconsistencey: mask path: {mask_name}. image path: {im_name}"
    mask_rle = rle_encode_less_memory(new_mask > 0.5)
    
    row_df_normalized["id"].append(new_id)
    row_df_normalized["rle"].append(mask_rle)
    row_df_normalized["organ"].append(new_organ)
    row_df_normalized["img_height"].append(new_mask.shape[0])
    row_df_normalized["img_width"].append(new_mask.shape[1])
    
    copyfile(
        im_name,
        os.path.join("/home/vova/data/exps/hubmap_2022/denis_add_data/v4/images", os.path.basename(im_name))
    )
    
row_df_normalized = pd.DataFrame(row_df_normalized)

In [ ]:
row_df_normalized

In [ ]:
row_df_normalized[row_df_normalized["organ"] == CLASSES[1]]

In [ ]:
row_df_normalized = pd.read_csv("/home/vova/data/exps/hubmap_2022/denis_add_data/v4/train.csv")

In [ ]:
len(row_df_normalized)

In [ ]:
idx = 117
for idx in range(80,140):
    img = imread_rgb(
        f"/home/vova/data/exps/hubmap_2022/denis_add_data/v4/images/{row_df_normalized.id.iloc[idx]}.tif",
        backend="tifi"
    )
    mask = rle_decode(row_df_normalized.rle.iloc[idx], shape=(row_df_normalized.iloc[idx].img_width, row_df_normalized.iloc[idx].img_height))
    # plt.figure(figsize=(10,10))
    # plt.title("Mask")
    # plt.imshow(mask)
    # plt.show()
    plt.figure(figsize=(10,10))
    plt.title("Mask Overlay")
    plt.imshow(overlay_mask(img, mask, color_id=0))
    plt.show()

In [ ]:
# !rm /home/vova/data/exps/hubmap_2022/denis_add_data/v4 -rf

In [ ]:
# row_df_normalized.to_csv("/home/vova/data/exps/hubmap_2022/denis_add_data/v4/train.csv", index=False)